In [72]:
# Import some useful packages
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, IntSlider, FloatSlider

import tensorflow as tf
import tensorflow.keras.backend as K

In [73]:
# Layers for FNN
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Lambda, concatenate
from tensorflow.keras.layers import Dense

# Optimizers for training
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras import metrics

# Losses for training
from tensorflow.keras import losses

# For data preprocessing
from tensorflow.keras import datasets
from tensorflow.keras.utils import to_categorical

In [74]:
# Load dataset
(X_train, y_train0), (X_test, y_test0) = datasets.mnist.load_data()


In [75]:
# Reshape sizeL:將 x 改成784維度
x_train = x_train.reshape(-1, 28*28)
x_test = x_test.reshape(-1, 28*28)

# Normalize the range of featurs
x_train = x_train / x_train.max()
x_test = x_test / x_test.max()

# One-hot encoding:將y轉換成類別資料
y_train = to_categorical(y_train0, 10)
y_test = to_categorical(y_test0, 10)

In [76]:
x_train.shape

(60000, 784)

In [77]:
enc_1 = Dense(512, activation='sigmoid')
enc_2 = Dense(128, activation='sigmoid')

enc_mean = Dense(2)
enc_log_var = Dense(2)

dec_2 = Dense(128, activation='sigmoid')
dec_1 = Dense(512, activation='sigmoid')
dec_0 = Dense(784, activation='sigmoid')


In [78]:
x = Input(shape=(784,))
enc_1_x = enc_1(x)
enc_2_x = enc_2(enc_1_x)


z_mean = enc_mean(enc_2_x)
z_log_var = enc_log_var(enc_2_x)


進行抽樣分配

In [79]:
def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean = 0 and std = 1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [80]:
# Sampling function wrapped as a Keras layer
z = Lambda(sampling, output_shape=(2,))([z_mean, z_log_var])

# Define Decoder part of VAE
z_input = Input(shape=(2,))
x_hat = dec_0(dec_1(dec_2(dec_3(z_input))))

In [81]:
x_hat = dec_0(dec_1(dec_2(dec_3(z))))

VAE = Model(x, x_hat)
VAE.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 784)]        0                                            
__________________________________________________________________________________________________
dense_32 (Dense)                (None, 512)          401920      input_11[0][0]                   
__________________________________________________________________________________________________
dense_33 (Dense)                (None, 128)          65664       dense_32[0][0]                   
__________________________________________________________________________________________________
dense_34 (Dense)                (None, 2)            258         dense_33[0][0]                   
____________________________________________________________________________________________

In [82]:
# reconstruction_loss = tf.keras.losses.mse(inputs, outputs)
reconstruction_loss = 784 * losses.mean_squared_error(x, x_hat)

kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)

VAE.add_loss(vae_loss)

In [83]:
VAE.compile(optimizer=Adam())

In [84]:
VAE.fit(x_train, x_train, batch_size=100, epochs=12)

ValueError: ('Error when checking model target: expected no data, but got:', array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]))

訓練VAE